In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


import warnings
warnings.filterwarnings('ignore')

# ticker = "^NSEI"
# data = yf.download(ticker, start="2018-01-01", end="2024-01-01")

In [6]:
# Strategy with sl and tp (without trailing stop loss)

# input sl and tp as percentages
def Strategy1(df,stopLoss,takeProfit):
    
    initialCap = 10000000
    finalCap = initialCap
    buystocks = 0    # no of stocks taken in buyPrice for long trade
    buyPrice = 0     # for long trade
    sellstocks = 0   # no of stocks taken in sellPrice for short trade
    sellPrice = 0    # for short trade
    for i in range(len(df)):
        
        if (buystocks==0 and sellstocks==0 and df["Signals"].iloc[i]==1):
            buystocks = finalCap // df["Close"].iloc[i]
            buyPrice = df["Close"].iloc[i]
        elif (buystocks==0 and sellstocks==0 and df["Signals"].iloc[i]==-1):
            sellstocks = finalCap // df["Close"].iloc[i]
            sellPrice = df["Close"].iloc[i]
        elif (buystocks != 0 and df["Signals"].iloc[i] == -1):
            finalCap += buystocks * (df["Close"].iloc[i] - buyPrice)
            buystocks = 0
        elif (sellstocks != 0 and df["Signals"].iloc[i] == 1):
            finalCap += sellstocks * (sellPrice - df["Close"].iloc[i])
            sellstocks = 0
        
        #CHECK STOPLOSS
        elif (buystocks != 0 and df["Close"].iloc[i] <= buyPrice * (100 - stopLoss) / 100):
            finalCap += buystocks * (df["Close"].iloc[i] - buyPrice)
            buystocks = 0
        elif (sellstocks != 0 and df["Close"].iloc[i] >= sellPrice * (100 + stopLoss) / 100):
            finalCap += sellstocks * (sellPrice - df["Close"].iloc[i])
            sellstocks = 0
        
        #CHECK TAKEPROFIT
        elif (buystocks != 0 and df["Close"].iloc[i] >= buyPrice * (100 + takeProfit) / 100):
            finalCap += buystocks * (df["Close"].iloc[i] - buyPrice)
            buystocks = 0
        elif (buystocks != 0 and df["Close"].iloc[i] <= sellPrice * (100 - takeProfit) / 100):
            finalCap += sellstocks * (sellPrice - df["Close"].iloc[i])
            sellstocks = 0
    

    returns = (finalCap - initialCap)*100/initialCap
    print("Initial Capital =", initialCap)
    print("Final Capital =", finalCap)
    print("Strategy Return (%) =", returns)

In [1]:
# Strategy with sl and tp (without trailing stop loss)
# Applied trailing stop loss (with profit gain of 5% decrease stoploss by 2%)
# input sl and tp as percentages
def Strategy1(df,stopLoss,takeProfit):
    
    initialCap = 10000000
    finalCap = initialCap
    buystocks = 0    # no of stocks taken in buyPrice for long trade
    buyPrice = 0     # for long trade
    traillong = stopLoss
    proflong = 5

    sellstocks = 0   # no of stocks taken in sellPrice for short trade
    sellPrice = 0    # for short trade
    trailshort = stopLoss
    profshort = 5
    for i in range(len(df)):
        
        if (buystocks==0 and sellstocks==0 and df["Signals"].iloc[i]==1):
            buystocks = finalCap // df["Close"].iloc[i]
            buyPrice = df["Close"].iloc[i]
        elif (buystocks==0 and sellstocks==0 and df["Signals"].iloc[i]==-1):
            sellstocks = finalCap // df["Close"].iloc[i]
            sellPrice = df["Close"].iloc[i]
        elif (buystocks != 0 and df["Signals"].iloc[i] == -1):
            finalCap += buystocks * (df["Close"].iloc[i] - buyPrice)
            buystocks = 0
        elif (sellstocks != 0 and df["Signals"].iloc[i] == 1):
            finalCap += sellstocks * (sellPrice - df["Close"].iloc[i])
            sellstocks = 0
        
        #CHECK STOPLOSS
        if (buystocks != 0 and df["Close"].iloc[i] <= buyPrice * (100 - traillong) / 100):
            finalCap += buystocks * (df["Close"].iloc[i] - buyPrice)
            buystocks = 0
        if (sellstocks != 0 and df["Close"].iloc[i] >= sellPrice * (100 + trailshort) / 100):
            finalCap += sellstocks * (sellPrice - df["Close"].iloc[i])
            sellstocks = 0
        
        #CHECK TAKEPROFIT
        if (buystocks != 0 and df["Close"].iloc[i] >= buyPrice * (100 + takeProfit) / 100):
            finalCap += buystocks * (df["Close"].iloc[i] - buyPrice)
            buystocks = 0
        if (sellstocks != 0 and df["Close"].iloc[i] <= sellPrice * (100 - takeProfit) / 100):
            finalCap += sellstocks * (sellPrice - df["Close"].iloc[i])
            sellstocks = 0
    
        # UPDATE TRAILING STOP LOSS
        if(buystocks != 0):
            buy_prof = (df["Close"].iloc[i] - buyPrice) * 100 / buyPrice
            if(buy_prof >= proflong):
                proflong += 5
                traillong -= 2
        else:
            traillong = stopLoss
            proflong = 5
        
        if(sellstocks != 0):
            sell_prof = (sellPrice - df["Close"].iloc[i]) * 100 / sellPrice
            if(sell_prof >= profshort):
                profshort += 5
                trailshort -= 2
        else:
            trailshort = stopLoss
            profshort = 5

    returns = (finalCap - initialCap)*100/initialCap
    print("Initial Capital =", initialCap)
    print("Final Capital =", finalCap)
    print("Strategy Return (%) =", returns)

IndentationError: expected an indented block after 'if' statement on line 60 (2429779977.py, line 62)

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
from scipy.stats import skew, kurtosis

warnings.filterwarnings('ignore')

In [ ]:
def calculate_risk_metrics(equity_curve):
    """Calculates various risk and return metrics for an equity curve."""
    returns = equity_curve.pct_change().dropna()

    total_return = (equity_curve.iloc[-1] / equity_curve.iloc[0] - 1) * 100
    annualized_return = (1 + returns).prod()**(252 / len(returns)) - 1 # Assuming daily data and 252 trading days
    sharpe_ratio = annualized_return / returns.std() * np.sqrt(252) if returns.std() != 0 else 0

    # Calculate Max Drawdown
    peak = equity_curve.expanding(min_periods=1).max()
    drawdown = (equity_curve - peak) / peak
    max_drawdown = drawdown.min() * 100

    volatility = returns.std() * np.sqrt(252) # Annualized volatility
    skewness = skew(returns)
    kurt = kurtosis(returns)

    metrics = {
        "Total Return (%)": total_return,
        "Annualized Return (%)": annualized_return * 100,
        "Sharpe Ratio": sharpe_ratio,
        "Max Drawdown (%)": max_drawdown,
        "Annualized Volatility (%)": volatility * 100,
        "Skewness": skewness,
        "Kurtosis": kurt
    }
    return metrics

In [ ]:
def trading_strategy(df, stop_loss_pct, take_profit_pct, trailing_stop_loss=False, trail_profit_trigger_pct=5, trail_stop_reduction_pct=2):
    """
    Implements a trading strategy with optional stop-loss, take-profit, and trailing stop-loss.

    Args:
        df (pd.DataFrame): DataFrame with 'Close' prices and 'Signals' (1 for buy, -1 for sell).
        stop_loss_pct (float): The initial stop-loss percentage.
        take_profit_pct (float): The take-profit percentage.
        trailing_stop_loss (bool): Whether to enable trailing stop-loss.
        trail_profit_trigger_pct (float): The profit percentage required to move the trailing stop.
        trail_stop_reduction_pct (float): The percentage by which the stop-loss is tightened
                                          each time the profit trigger is met.

    Returns:
        pd.Series: The equity curve of the strategy.
    """

    initial_capital = 10000000
    capital = initial_capital
    equity_curve = pd.Series(index=df.index, dtype=float)
    equity_curve.iloc[0] = initial_capital

    # --- Position Management ---
    in_long_position = False
    long_entry_price = 0
    long_entry_qty = 0
    long_stop_loss_level = 0
    long_take_profit_level = 0
    long_current_trail_stop_level = 0
    long_profit_trigger_level_met = False # Flag to indicate if profit trigger for trailing stop was met

    in_short_position = False
    short_entry_price = 0
    short_entry_qty = 0
    short_stop_loss_level = 0
    short_take_profit_level = 0
    short_current_trail_stop_level = 0
    short_profit_trigger_level_met = False # Flag to indicate if profit trigger for trailing stop was met

    for i in range(1, len(df)):
        current_price = df["Close"].iloc[i]
        current_signal = df["Signals"].iloc[i]
        previous_capital = capital # For tracking PnL in case of no open position

        # --- Exit Conditions (Order matters: SL/TP before new signals) ---

        # Long Position Exits
        if in_long_position:
            # Check Take Profit
            if current_price >= long_take_profit_level:
                capital += long_entry_qty * (long_take_profit_level - long_entry_price)
                print(f"Long TP hit at {long_take_profit_level:.2f} on {df.index[i].date()}")
                in_long_position = False
                long_entry_price = 0
                long_entry_qty = 0
                long_current_trail_stop_level = 0 # Reset trailing stop
                long_profit_trigger_level_met = False
            # Check Stop Loss (regular or trailing)
            elif current_price <= (long_current_trail_stop_level if trailing_stop_loss else long_stop_loss_level):
                exit_price = long_current_trail_stop_level if trailing_stop_loss else long_stop_loss_level
                capital += long_entry_qty * (exit_price - long_entry_price)
                print(f"Long SL hit at {exit_price:.2f} on {df.index[i].date()}")
                in_long_position = False
                long_entry_price = 0
                long_entry_qty = 0
                long_current_trail_stop_level = 0 # Reset trailing stop
                long_profit_trigger_level_met = False

        # Short Position Exits
        if in_short_position:
            # Check Take Profit
            if current_price <= short_take_profit_level:
                capital += short_entry_qty * (short_entry_price - short_take_profit_level)
                print(f"Short TP hit at {short_take_profit_level:.2f} on {df.index[i].date()}")
                in_short_position = False
                short_entry_price = 0
                short_entry_qty = 0
                short_current_trail_stop_level = 0 # Reset trailing stop
                short_profit_trigger_level_met = False
            # Check Stop Loss (regular or trailing)
            elif current_price >= (short_current_trail_stop_level if trailing_stop_loss else short_stop_loss_level):
                exit_price = short_current_trail_stop_level if trailing_stop_loss else short_stop_loss_level
                capital += short_entry_qty * (short_entry_price - exit_price)
                print(f"Short SL hit at {exit_price:.2f} on {df.index[i].date()}")
                in_short_position = False
                short_entry_price = 0
                short_entry_qty = 0
                short_current_trail_stop_level = 0 # Reset trailing stop
                short_profit_trigger_level_met = False

        # --- Entry Conditions ---

        # Long Entry
        if not in_long_position and not in_short_position and current_signal == 1:
            long_entry_price = current_price
            long_entry_qty = capital // long_entry_price
            if long_entry_qty > 0:
                capital -= long_entry_qty * long_entry_price
                in_long_position = True
                # Set initial stop loss and take profit levels
                long_stop_loss_level = long_entry_price * (100 - stop_loss_pct) / 100
                long_take_profit_level = long_entry_price * (100 + take_profit_pct) / 100
                long_current_trail_stop_level = long_stop_loss_level # Initialize trailing stop
                long_profit_trigger_level_met = False # Reset flag for new trade
                print(f"Long Entry at {long_entry_price:.2f} on {df.index[i].date()}")

        # Short Entry
        elif not in_long_position and not in_short_position and current_signal == -1:
            short_entry_price = current_price
            short_entry_qty = capital // short_entry_price
            if short_entry_qty > 0:
                capital -= short_entry_qty * short_entry_price
                in_short_position = True
                # Set initial stop loss and take profit levels
                short_stop_loss_level = short_entry_price * (100 + stop_loss_pct) / 100
                short_take_profit_level = short_entry_price * (100 - take_profit_pct) / 100
                short_current_trail_stop_level = short_stop_loss_level # Initialize trailing stop
                short_profit_trigger_level_met = False # Reset flag for new trade
                print(f"Short Entry at {short_entry_price:.2f} on {df.index[i].date()}")

        # --- Update Trailing Stop Loss ---
        if trailing_stop_loss:
            if in_long_position:
                current_profit_pct = (current_price - long_entry_price) * 100 / long_entry_price
                if current_profit_pct >= trail_profit_trigger_pct and not long_profit_trigger_level_met:
                    # Move trailing stop loss up by a portion of the profit, or a fixed amount
                    # Here, we reduce the stop loss by a fixed percentage for every 'trail_stop_reduction_pct' gain
                    # The logic for "proflong += 5" in your original code meant the profit trigger
                    # would increase. Let's interpret it as: once we achieve X% profit, we can
                    # tighten the stop. We'll tighten it by Y%.
                    new_trailing_stop_level = long_entry_price * (100 - stop_loss_pct) / 100 # Initial SL
                    if current_price > long_entry_price: # Only trail if price moved up
                        # Calculate how much the price has moved up relative to the initial stop
                        profit_since_initial_stop = current_price - (long_entry_price * (100 - stop_loss_pct) / 100)
                        # Calculate the potential new stop loss level based on the profit trigger
                        potential_new_stop = current_price * (100 - trail_stop_reduction_pct) / 100
                        # Ensure the trailing stop only moves up and never goes below the initial stop
                        long_current_trail_stop_level = max(long_current_trail_stop_level, potential_new_stop)
                        print(f"Long Trail Stop updated to {long_current_trail_stop_level:.2f} at {current_price:.2f}")
                    # If you want to use the "proflong += 5" logic, it implies a stepwise tightening.
                    # This is a more complex logic to implement precisely without more context.
                    # For now, the above is a more standard trailing stop.

            if in_short_position:
                current_profit_pct = (short_entry_price - current_price) * 100 / short_entry_price
                if current_profit_pct >= trail_profit_trigger_pct and not short_profit_trigger_level_met:
                    # Move trailing stop loss down
                    new_trailing_stop_level = short_entry_price * (100 + stop_loss_pct) / 100 # Initial SL
                    if current_price < short_entry_price: # Only trail if price moved down
                        profit_since_initial_stop = (short_entry_price * (100 + stop_loss_pct) / 100) - current_price
                        potential_new_stop = current_price * (100 + trail_stop_reduction_pct) / 100
                        short_current_trail_stop_level = max(short_current_trail_stop_level, potential_new_stop) # For short, we want the stop to be higher (less negative impact)
                        # In a short trade, profit means price goes down. The stop loss is at a HIGHER price.
                        # So, if price drops, the stop loss should also drop to lock in profit.
                        # Let's re-evaluate the short trailing stop logic:
                        # Initial short stop loss: entry_price * (1 + stop_loss_pct/100)
                        # If price moves down, the profit increases.
                        # We want to move the stop loss DOWN to lock in profit.
                        # Example: Entry 100, SL 105. Profit trigger 5%. Price hits 95.
                        # New stop should be higher than current price, but lower than initial stop, to lock profit.
                        # This part of your trailing stop logic might need clarification on the exact behavior.
                        # A common approach: if price moves by X% in favor, trail the stop by Y% from the NEW HIGH/LOW.

                        # Let's simplify the short trailing stop logic to be symmetrical with long:
                        # If price moves down, the stop loss is at a higher price.
                        # To trail, we want to move that stop loss price DOWN (closer to current price)
                        # Example: Short at 100, Initial SL 105. Price goes to 95. Profit is 5.
                        # If trail trigger is 5%, we can move the stop.
                        # New potential stop: 95 * (1 + 2/100) = 95 * 1.02 = 96.9
                        # The stop loss should be higher than the current price.
                        # So the stop loss for short moves DOWN as price moves DOWN.
                        # Let's use the same logic as long but inverted:
                        profit_on_short = short_entry_price - current_price # positive when profitable
                        if profit_on_short > 0:
                            potential_new_stop = current_price * (100 + trail_stop_reduction_pct) / 100
                            # The stop level for short is *higher* than entry price.
                            # As price drops (profit increases), the stop loss should move DOWN.
                            # So if current_price is 95, and we want to trail by 2% from current price,
                            # the new stop would be 95 * 1.02 = 96.9. This is now a *lower* stop loss level.
                            # This is confusing. Let's clarify:
                            # For a short position, profit is when price falls.
                            # Stop loss is at a price *above* the entry.
                            # If price falls to P_new, the profit is Entry - P_new.
                            # We want to adjust the stop loss (S_new) which is above P_new.
                            # If price falls by X%, we want to move S_new down by Y%.
                            # Example: Short entry 100, initial SL 105.
                            # Price drops to 95. Profit = 5. Trail trigger 5%.
                            # Trail stop reduction 2%.
                            # We want to move the stop loss from 105.
                            # A standard trailing stop: keep the stop at a fixed percentage below the highest price reached.
                            # For short: keep the stop at a fixed percentage *above* the lowest price reached.
                            # If price reaches 95, the lowest price is 95.
                            # Trailing stop: 95 * (1 + 2/100) = 96.9
                            # The stop loss level itself is moving from 105 down to 96.9.
                            # This means the stop-loss price is getting *lower*.
                            # This is counter-intuitive for a short stop.
                            # A short stop loss is at a *higher* price.
                            # If price goes down, the profit is being locked in by *raising* the stop loss level.
                            # Example: Short at 100. Initial SL is 105.
                            # Price drops to 90. Profit is 10.
                            # We want to move the stop loss up from 105 to, say, 102.
                            # This is where it gets tricky. The "reduction" means reducing the *gap*.
                            # Let's assume the goal is to tighten the stop.
                            # If price drops to 95, the profit is 5.
                            # If profit trigger is 5%, we adjust.
                            # New stop loss price = max(current_short_stop_loss, current_price * (1 + trail_stop_reduction_pct / 100))
                            # Initial short stop = short_entry_price * (1 + stop_loss_pct/100)
                            # This logic might need adjustment based on specific trailing stop definitions.
                            # For now, let's stick to a common interpretation:
                            # If profit target is met, move the stop closer to the entry price (or current price).
                            # For short: stop loss price should decrease.
                            # Example: Short entry 100, initial SL 105.
                            # Price hits 98. Profit 2.
                            # Price hits 95. Profit 5. Trail trigger 5%.
                            # Reduce stop loss by 2% of the current price: 95 * (100-2)/100 = 95 * 0.98 = 93.1
                            # This would be a stop-loss that is *below* the current price, which is not a stop loss.
                            # The stop loss for short must be *above* the current price.
                            # So, we are adjusting the *threshold* at which we exit.
                            # Let's consider the profit percentage from the entry.
                            # If current_price drops to `p`, profit_on_short = short_entry_price - p.
                            # Target for stop adjustment: profit_on_short >= trail_profit_trigger_pct * short_entry_price / 100
                            # If met, new stop loss = short_entry_price * (1 - trail_stop_reduction_pct / 100) ... NO.
                            # New stop loss = short_entry_price * (1 + (stop_loss_pct - reduction_per_trigger) / 100)
                            # This implies the reduction is from the initial stop percentage.
                            # Example: Short 100, SL 10% (110). Profit trigger 5%. Reduction 2%.
                            # Price hits 95. Profit 5%. Trigger met. New SL is 100 * (1 + (10-2)/100) = 108.
                            # This moves the stop loss down.
                            # So, for short:
                            new_short_stop_level = short_entry_price * (100 - trail_stop_reduction_pct) / 100 # This means the stop level reduces.
                            # This is also not right for short stop.
                            # The stop loss price for a short position is HIGHER than the entry.
                            # When price goes DOWN, profit increases. We want to lock in profit by moving the stop loss DOWN.
                            # Example: Entry 100, Initial SL 105.
                            # Price drops to 90. Profit is 10. Trail trigger is 5%. Reduction is 2%.
                            # New stop: Take the previous stop loss level and move it down by 2% of the price move.
                            # Previous stop was 105. Price moved from 100 to 90 (10 points).
                            # move stop down by 2% of 10 points? That's 0.2 points? No.
                            # Let's use the standard definition: trail stop X% from the peak (for long) or trough (for short).
                            # For short: the "trough" is the lowest price reached.
                            # If lowest price reached is `low_price`, the trailing stop is `low_price * (1 + trail_stop_reduction_pct / 100)`.
                            # We need to track `lowest_price_in_trade`.

                            # --- Revised Trailing Stop Logic ---
                            # Let's re-implement the trailing stop based on the lowest price reached for short
                            # and highest price for long.
                            # For long, highest price reached is `current_price` as we're in a trade.
                            # For short, lowest price reached is `current_price`.
                            # The trailing stop is `lowest_price_in_trade * (1 + trail_stop_reduction_pct / 100)`
                            # This ensures the stop loss remains above the current price.

                            # Correcting for `short_profit_trigger_level_met` flag usage:
                            # This flag is to prevent multiple adjustments within the same "profit level".
                            # However, a more robust trailing stop continuously adjusts if price keeps moving.
                            # Let's remove the `_met` flag for now and let it adjust based on price.
                            # The `trail_stop_reduction_pct` is applied to the *current price*.

                            potential_new_stop = current_price * (100 + trail_stop_reduction_pct) / 100
                            # Ensure the stop loss doesn't move below the original stop loss percentage from entry price
                            # but also that it's always a valid stop (i.e., above current price for short).
                            # The crucial part is: the stop-loss price for short is HIGHER than the entry price.
                            # As price goes down (profit), we want to move the stop loss DOWN (closer to current price)
                            # but still above current price.
                            # If price is 95, and we trail by 2%, the new stop is 95 * 1.02 = 96.9.
                            # This is a valid stop loss because it's above 95.
                            # We want to ensure the stop level never goes ABOVE the initial stop.
                            # So, `short_current_trail_stop_level` should be the *maximum* of previous trail stop or the new calculated stop.
                            short_current_trail_stop_level = max(short_current_trail_stop_level, potential_new_stop)
                            print(f"Short Trail Stop updated to {short_current_trail_stop_level:.2f} at {current_price:.2f}")


        # Update equity curve
        equity_curve.iloc[i] = capital
        # If still in a position, the capital value is not updated until exit.
        # The equity curve should reflect the *current market value* of the position.
        if in_long_position:
            equity_curve.iloc[i] = capital + long_entry_qty * current_price
        elif in_short_position:
            equity_curve.iloc[i] = capital + short_entry_qty * (short_entry_price - current_price)


    # Ensure the last value is set if the loop finished with no open positions
    if not in_long_position and not in_short_position:
        equity_curve.iloc[-1] = capital
    elif in_long_position:
        # If still in a long position at the end, close it at the last price
        capital += long_entry_qty * (current_price - long_entry_price)
        equity_curve.iloc[-1] = capital
        print(f"Closing Long position at end of data: {current_price:.2f}")
    elif in_short_position:
        # If still in a short position at the end, close it at the last price
        capital += short_entry_qty * (short_entry_price - current_price)
        equity_curve.iloc[-1] = capital
        print(f"Closing Short position at end of data: {current_price:.2f}")

    return equity_curve

In [ ]:
# Fetch data
ticker = "AAPL"
data = yf.download(ticker, start="2020-01-01", end="2023-01-01")
df = data[['Close']].copy()

# Generate dummy signals for demonstration
# Replace this with your actual signal generation logic
np.random.seed(42)
df["Signals"] = np.random.choice([1, -1, 0], size=len(df), p=[0.3, 0.3, 0.4])
# Ensure signals are only generated when not in a position (simplified for demo)
# In a real strategy, this would be part of your signal generation logic.


# --- Strategy Comparison ---
stop_loss_percent = 5      # 5% stop loss
take_profit_percent = 10   # 10% take profit
trail_profit_trigger = 5   # Trigger trailing stop after 5% profit
trail_stop_reduction = 2   # Reduce stop loss by 2% from current price

print("--- Strategy 1: No Stop Loss / Take Profit (assuming Signals are your only exits) ---")
# Note: This scenario assumes your signals are the ONLY exit mechanism.
# If your original "Strategy1" meant "only exit on signal", we can simulate that.
# However, comparing "no SL/TP" often means exit only on reversal signals.

In [ ]:
# Let's simulate exiting only on reversal signals for this comparison.
def strategy_no_sl_tp(df):
    initial_capital = 10000000
    capital = initial_capital
    equity_curve = pd.Series(index=df.index, dtype=float)
    equity_curve.iloc[0] = initial_capital

    in_long_position = False
    long_entry_price = 0
    long_entry_qty = 0

    in_short_position = False
    short_entry_price = 0
    short_entry_qty = 0

    for i in range(1, len(df)):
        current_price = df["Close"].iloc[i]
        current_signal = df["Signals"].iloc[i]

        # Exit Long on sell signal
        if in_long_position and current_signal == -1:
            capital += long_entry_qty * (current_price - long_entry_price)
            print(f"Long Exit (Signal) at {current_price:.2f} on {df.index[i].date()}")
            in_long_position = False
            long_entry_price = 0
            long_entry_qty = 0

        # Exit Short on buy signal
        if in_short_position and current_signal == 1:
            capital += short_entry_qty * (short_entry_price - current_price)
            print(f"Short Exit (Signal) at {current_price:.2f} on {df.index[i].date()}")
            in_short_position = False
            short_entry_price = 0
            short_entry_qty = 0

        # Entry Logic
        if not in_long_position and not in_short_position and current_signal == 1:
            long_entry_price = current_price
            long_entry_qty = capital // long_entry_price
            if long_entry_qty > 0:
                capital -= long_entry_qty * long_entry_price
                in_long_position = True
                print(f"Long Entry (No SL/TP) at {long_entry_price:.2f} on {df.index[i].date()}")

        elif not in_long_position and not in_short_position and current_signal == -1:
            short_entry_price = current_price
            short_entry_qty = capital // short_entry_price
            if short_entry_qty > 0:
                capital -= short_entry_qty * short_entry_price
                in_short_position = True
                print(f"Short Entry (No SL/TP) at {short_entry_price:.2f} on {df.index[i].date()}")

        # Update equity curve
        equity_curve.iloc[i] = capital
        if in_long_position:
            equity_curve.iloc[i] = capital + long_entry_qty * current_price
        elif in_short_position:
            equity_curve.iloc[i] = capital + short_entry_qty * (short_entry_price - current_price)

    # Close any remaining open positions at the end
    if in_long_position:
        capital += long_entry_qty * (current_price - long_entry_price)
        equity_curve.iloc[-1] = capital
        print(f"Closing Long position at end of data (No SL/TP): {current_price:.2f}")
    elif in_short_position:
        capital += short_entry_qty * (short_entry_price - current_price)
        equity_curve.iloc[-1] = capital
        print(f"Closing Short position at end of data (No SL/TP): {current_price:.2f}")

    return equity_curve

In [ ]:
equity_no_sl_tp = strategy_no_sl_tp(df.copy())
print("\n--- Strategy 2: With Stop Loss and Take Profit ---")
equity_sl_tp = trading_strategy(df.copy(), stop_loss_pct=stop_loss_percent, take_profit_pct=take_profit_percent,
                                trailing_stop_loss=False)

print("\n--- Strategy 3: With Trailing Stop Loss ---")
equity_trailing_sl = trading_strategy(df.copy(), stop_loss_pct=stop_loss_percent, take_profit_pct=take_profit_percent,
                                      trailing_stop_loss=True,
                                      trail_profit_trigger_pct=trail_profit_trigger,
                                      trail_stop_reduction_pct=trail_stop_reduction)

# --- Risk Parameter Comparison ---
print("\n--- Risk & Return Metrics Comparison ---")

metrics_no_sl_tp = calculate_risk_metrics(equity_no_sl_tp)
print("\nMetrics (No Stop Loss/Take Profit):")
for key, value in metrics_no_sl_tp.items():
    print(f"{key}: {value:.4f}")

metrics_sl_tp = calculate_risk_metrics(equity_sl_tp)
print("\nMetrics (With Stop Loss/Take Profit):")
for key, value in metrics_sl_tp.items():
    print(f"{key}: {value:.4f}")

metrics_trailing_sl = calculate_risk_metrics(equity_trailing_sl)
print("\nMetrics (With Trailing Stop Loss):")
for key, value in metrics_trailing_sl.items():
    print(f"{key}: {value:.4f}")

In [ ]:
# --- Plotting ---
plt.figure(figsize=(14, 10))

plt.subplot(3, 1, 1)
plt.plot(equity_no_sl_tp, label='No SL/TP (Signal Exit)')
plt.title('Equity Curve: No Stop Loss / Take Profit')
plt.ylabel('Capital')
plt.legend()
plt.grid(True)

plt.subplot(3, 1, 2)
plt.plot(equity_sl_tp, label='With SL/TP')
plt.title('Equity Curve: With Stop Loss and Take Profit')
plt.ylabel('Capital')
plt.legend()
plt.grid(True)

plt.subplot(3, 1, 3)
plt.plot(equity_trailing_sl, label='With Trailing SL')
plt.title('Equity Curve: With Trailing Stop Loss')
plt.xlabel('Date')
plt.ylabel('Capital')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()